# load libraries and data directory

In [31]:
suppressMessages(library(RankProd)) 
suppressMessages(library(limma))
library(gplots)
library(plotrix)
library(RColorBrewer)
library(methods)
library(edgeR)
library(ggbiplot)
library(genefilter)

suppressMessages(library(gtools)) 
suppressMessages(library(Hmisc))
suppressMessages(library(minet))
suppressMessages(library(reshape2))
suppressMessages(library(plyr))
suppressMessages(library(outliers))
suppressMessages(library(magrittr))
suppressMessages(library(dplyr))
suppressMessages(library(tidyr))
suppressMessages(library(purrr))
suppressMessages(library(stringr))

library(tibble)
library(readr)

options(stringsAsFactors = FALSE)

In [32]:
mRNA_data_dir = '~/data/GSK_enfisema/arrays mRNA'
pheno_csv = '~/data/GSK_enfisema/pheno_groups.csv'
working_dir = '~/Documents/GSK_enfisema/'

# read U219 array cell files and RMA normalize

## create phenotypic data_frame

In [37]:
groups = read.csv(pheno_csv,header = TRUE)

fi = list.files(mRNA_data_dir)[grepl('.CEL',list.files(mRNA_data_dir))]

pheno = data_frame(sample = fi) %>% rowwise %>% 
  mutate(patient = strsplit(sample,split='_')[[1]][1],      
         pat = gsub('RF-','',patient),
         pat = gsub('RF','',pat),
         pat = gsub('-1','',pat),
         pat = gsub('-2','',pat),
         pat = gsub('-3','',pat)) %>% ungroup %>% select(-patient)

pheno$pat[pheno$pat %in% groups$pat]
pheno$pat[!pheno$pat %in% groups$pat]
groups$pat[!groups$pat %in% pheno$pat]

pheno %<>% inner_join(groups) 
write_csv(pheno, str_c(working_dir,'pheno.csv'))

[1] "101" "102" "103" "104" "106" "107" "108" "109" "10"  "110" "111" "112"
[13] "113" "115" "117" "118" "12"  "16"  "17"  "1"   "20"  "30"  "31"  "32" 
[25] "42"  "44"  "49"  "4"   "50"  "55"  "59"  "5"   "62"  "66"  "67"  "79" 
[37] "80"  "81"  "85D"

[1] "114" "116" "119" "120" "122" "13"  "14"  "15"  "21"  "22"  "24"  "25" 
[13] "27"  "28"  "29"  "2"   "33"  "34"  "35"  "37"  "38"  "39"  "3"   "40" 
[25] "41"  "43"  "45"  "47"  "48"  "51"  "52"  "53"  "54"  "56"  "57"  "58" 
[37] "60"  "61"  "63"  "64"  "65"  "68"  "6"   "71"  "73"  "74"  "75"  "78" 
[49] "82"  "83"  "84D" "86D" "87"  "8"   "9"

[1] "105"

Joining, by = "pat"
